<a href="https://colab.research.google.com/github/syt73/python/blob/main/pythonbasic12_Machine_Learning_%ED%83%80%EC%9D%B4%ED%83%80%EB%8B%892.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# G 드라이브 연결
from google.colab import drive
drive.mount('/gdrive/')

# 모듈 설정
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns


# https://dacon.io/competitions/open/235539/data

train = pd.read_csv('/gdrive/My Drive/Colab Notebooks/titanic/train.csv')
test = pd.read_csv('/gdrive/My Drive/Colab Notebooks/titanic/test.csv')
submission = pd.read_csv('/gdrive/My Drive/Colab Notebooks/titanic/submission.csv')


# 데이터 결합 (위 아래 방향으로 결합)

# 분석을 끝내고 학습을 시킬때 다시 분리 하기 편하도록 split  열을 만들어 각각 이름을 부여한다. 
train['Split'] = 'Train'
test['Split'] = 'Test'

data = pd.concat([train, test], axis=0)

data_num = data.loc[:,['Pclass','Age','SibSp','Parch','Fare','Survived']] 

# age  평균
data_num['Age'] = data_num['Age'].fillna(data_num['Age'].mean())

# fare 최빈
data_num['Fare'] = data_num['Fare'].fillna(data_num['Fare'].mode()[0])



Mounted at /gdrive/


In [2]:
# 피처 엔지니어링 - 각 피처 열을 모델 학습에 맞게 전처리하고 data['Sex']
data['Sex']

0        male
1      female
2      female
3      female
4        male
        ...  
413      male
414    female
415      male
416      male
417      male
Name: Sex, Length: 1309, dtype: object

In [3]:
# female 0  male 1

data.loc[data['Sex']=='female','Sex'] = 0
data.loc[data['Sex']=='male','Sex'] = 1

data['Sex']

0      1
1      0
2      0
3      0
4      1
      ..
413    1
414    0
415    1
416    1
417    1
Name: Sex, Length: 1309, dtype: object

In [4]:
data['Sex'] = data['Sex'].astype(int)
data['Sex']

0      1
1      0
2      0
3      0
4      1
      ..
413    1
414    0
415    1
416    1
417    1
Name: Sex, Length: 1309, dtype: int64

In [5]:
# Name
data['Name']

0                                Braund, Mr. Owen Harris
1      Cumings, Mrs. John Bradley (Florence Briggs Th...
2                                 Heikkinen, Miss. Laina
3           Futrelle, Mrs. Jacques Heath (Lily May Peel)
4                               Allen, Mr. William Henry
                             ...                        
413                                   Spector, Mr. Woolf
414                         Oliva y Ocana, Dona. Fermina
415                         Saether, Mr. Simon Sivertsen
416                                  Ware, Mr. Frederick
417                             Peter, Master. Michael J
Name: Name, Length: 1309, dtype: object

In [6]:
title_name = data['Name'].str.split(", ", expand=True)[1]

# 띄어 쓰기 주의...점 뒤에 한칸 있음

In [7]:
   # 시리즈 데이터형을 str로 변환하여 자르고 인덱싱
title = title_name.str.split(".", expand=True)[0]
title.value_counts(dropna=False)

Mr              757
Miss            260
Mrs             197
Master           61
Rev               8
Dr                8
Col               4
Mlle              2
Major             2
Ms                2
Lady              1
Sir               1
Mme               1
Don               1
Capt              1
the Countess      1
Jonkheer          1
Dona              1
Name: 0, dtype: int64

In [8]:
# 비슷한 속성을 가진 타이틀 들을 그룹으로 묶자. 

title = title.replace(['Ms'], 'Miss')
title = title.replace(['Mlle', 'the Countess', 'Lady', 'Don', 'Dona', 'Mme', 'Sir', 'Jonkheer'], 'Noble')
title = title.replace(['Col', 'Major', 'Capt'], 'Officer')
title = title.replace(['Dr', 'Rev'], 'Priest')
data['Title'] = np.array(title)
data['Title'].value_counts(dropna=False)

Mr         757
Miss       262
Mrs        197
Master      61
Priest      16
Noble        9
Officer      7
Name: Title, dtype: int64

In [9]:
data

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Split,Title
0,1,0.0,3,"Braund, Mr. Owen Harris",1,22.0,1,0,A/5 21171,7.2500,NaN,S,Train,Mr
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.0,1,0,PC 17599,71.2833,C85,C,Train,Mrs
2,3,1.0,3,"Heikkinen, Miss. Laina",0,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Train,Miss
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.0,1,0,113803,53.1000,C123,S,Train,Mrs
4,5,0.0,3,"Allen, Mr. William Henry",1,35.0,0,0,373450,8.0500,NaN,S,Train,Mr
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,NaN,3,"Spector, Mr. Woolf",1,NaN,0,0,A.5. 3236,8.0500,NaN,S,Test,Mr
414,1306,NaN,1,"Oliva y Ocana, Dona. Fermina",0,39.0,0,0,PC 17758,108.9000,C105,C,Test,Noble
415,1307,NaN,3,"Saether, Mr. Simon Sivertsen",1,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S,Test,Mr
416,1308,NaN,3,"Ware, Mr. Frederick",1,NaN,0,0,359309,8.0500,NaN,S,Test,Mr


In [10]:
# Title 열을 만들었으므로 필요 없어진 name 열은 삭제한다. 

data= data.drop('Name', axis=1)
data

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Split,Title
0,1,0.0,3,1,22.0,1,0,A/5 21171,7.2500,NaN,S,Train,Mr
1,2,1.0,1,0,38.0,1,0,PC 17599,71.2833,C85,C,Train,Mrs
2,3,1.0,3,0,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Train,Miss
3,4,1.0,1,0,35.0,1,0,113803,53.1000,C123,S,Train,Mrs
4,5,0.0,3,1,35.0,0,0,373450,8.0500,NaN,S,Train,Mr
...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,NaN,3,1,NaN,0,0,A.5. 3236,8.0500,NaN,S,Test,Mr
414,1306,NaN,1,0,39.0,0,0,PC 17758,108.9000,C105,C,Test,Noble
415,1307,NaN,3,1,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S,Test,Mr
416,1308,NaN,3,1,NaN,0,0,359309,8.0500,NaN,S,Test,Mr
